<h1>Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [1]:
%reload_ext watermark
%reload_ext autoreload
%autoreload 2
# %watermark -p numpy,sklearn,pandas
# %watermark -p ipywidgets,cv2,PIL,matplotlib,plotly,netron
# %watermark -p torch,torchvision,torchaudio
# %watermark -p tensorflow,tensorboard,tflite
# %watermark -p onnx,tf2onnx,onnxruntime,tensorrt,tvm
# %matplotlib inline
# %config InlineBackend.figure_format='retina'
%config IPCompleter.use_jedi = False

# from IPython.display import display, Markdown, HTML, IFrame, Image, Javascript
# from IPython.core.magic import register_line_cell_magic, register_line_magic, register_cell_magic
# display(HTML('<style>.container { width:%d%% !important; }</style>' % 90))

import sys, os, io, logging, time, random, math
import json, base64, requests, shutil
import argparse, shlex, signal
import numpy as np

argparse.ArgumentParser.exit = lambda *arg, **kwargs: _IGNORE_

def _IMPORT(x):
    try:
        x = x.strip()
        if x[0] == '/' or x[0:2] == './':
            with open(x) as fr:
                x = fr.read()
        elif 'github' in x or 'gitee' in x:
            if x.startswith('import '):
                x = x[7:]
            if x.startswith('https://'):
                x = x[8:]
            if not x.endswith('.py'):
                x = x + '.py'
            x = x.replace('blob/main/', '').replace('blob/master/', '')
            if x.startswith('raw.githubusercontent.com'):
                x = 'https://' + x
                x = requests.get(x)
                if x.status_code == 200:
                    x = x.text
            elif x.startswith('github.com'):
                x = x.replace('github.com', 'raw.githubusercontent.com')
                mod = x.split('/')
                for s in ['/main/', '/master/']:
                    x = 'https://' + '/'.join(mod[:3]) + s + '/'.join(mod[-3:])
                    x = requests.get(x)
                    if x.status_code == 200:
                        x = x.text
                        break
            elif x.startswith('gitee.com'):
                mod = x.split('/')
                for s in ['/raw/main/', '/raw/master/']:
                    x = 'https://' + '/'.join(mod[:3]) + s + '/'.join(mod[3:])
                    x = requests.get(x)
                    if x.status_code == 200:
                        x = x.text
                        break
        exec(x, globals())
    except Exception as err:
        sys.stderr.write(f'request {x} : {err}')

def _DIR(x, dumps=True, ret=True):
    attrs = sorted([y for y in dir(x) if not y.startswith('_')])
    result = '%s: %s' % (str(type(x))[8:-2], json.dumps(attrs) if dumps else attrs)
    if ret:
        return result
    print(result)


In [33]:


import abc, sys
import threading

from multiprocessing import Queue
from queue import Empty
from enum import IntEnum, unique


class SingletonType(type):
    _instance_lock = threading.Lock()

    def __call__(cls, *args, **kwargs):
        if not hasattr(cls, "_instance"):
            with SingletonType._instance_lock:
                if not hasattr(cls, "_instance"):
                    cls._instance = super(SingletonType,cls).__call__(*args, **kwargs)
        return cls._instance


@unique
class MessageType(IntEnum):
    NOP = -1
    LOG = 1
    STATE = 2
    QUIT = 99


class Message(object):
    def __init__(self, what, arg1, arg2, obj):
        self.what = what
        self.arg1 = arg1
        self.arg2 = arg2
        self.obj = obj

    def __str__(self):
        obj = self.obj[:32] if isinstance(self.obj, str) else self.obj.__class__.__name__
        return f'Message({self.what}, {self.arg1}, {self.arg2}, {obj})'

    @staticmethod
    def obtain(what, arg1, arg2, obj):
        return Message(what, arg1, arg2, obj)


class MessageHandler(metaclass=abc.ABCMeta):
    def __init__(self, keys=[]):
        self.keys = keys
        self.mq = None

    @abc.abstractmethod
    def handle_message(self, what, arg1, arg2, obj):
        pass

    def send_message(self, what, arg1=-1, arg2=-1, obj=None):
        if self.mq:
            msg = Message.obtain(what, arg1, arg2, obj)
            self.mq.put(msg)
        return True

    def dispatch_message(self, msg):
        try:
            return self.handle_message(msg.what, msg.arg1, msg.arg2, msg.obj)
        except Exception as err:
            sys.stderr.write(f'{err}\n')


class DefaultHandler(MessageHandler):
    def __init__(self, mq):
        super(DefaultHandler, self).__init__(keys=[])
        self.mq = mq

    def handle_message(self, what, arg1, arg2, obj):
        return False


class MainLooper(threading.Thread, metaclass=SingletonType):

    def __init__(self):
        super(MainLooper, self).__init__(name='MainLooper')
        self.mq = Queue()
        self.handlers = {}
        self.H = DefaultHandler(self.mq)

    @property
    def default_handler(self):
        return self.H

    def add_handler(self, handler):
        handler.mq = self.mq
        for ty in handler.keys:
            if ty not in self.handlers:
                self.handlers[ty] = []
            self.handlers[ty].append(handler)

    def run(self):
        while True:
            try:
                msg = self.mq.get(timeout=3)
                if msg.what == MessageType.QUIT:
                    break
                if msg.what not in self.handlers:
                    continue
                for handler in self.handlers[msg.what]:
                    if handler.dispatch_message(msg):
                        break
            except Empty:
                pass


In [26]:
_IMPORT('/data/nb_easy/easy_utils.py')

In [27]:
MessageHandler.logger = nbeasy_get_logger('nbeasy')

In [29]:
@unique
class ServiceType(IntEnum):
    NOP = -1
    APP = 1
    SRS = 2


@unique
class StateType(IntEnum):
    NOP = -1
    RUNNING = 1
    STARTING = 2
    STARTED = 3
    STOPPING = 4
    STOPPED = 5
    STOPPTIMEOUT = 6
    CRASHED = 99
    

class ServiceStateMessageHandler(MessageHandler):
    keys = [MessageType.STATE]

    def __init__(self):
        super().__init__(keys=self.keys)
        
    def on_app(self, arg2, obj):
        if StateType.RUNNING == arg2:
            self.logger.info('app running')
        return True

    def on_srs(self, arg2, obj):
        if StateType.RUNNING == arg2:
            self.logger.info('srs running')
        return True
        
    def handle_message(self, what, arg1, arg2, obj):
        self.logger.info(f'({what}, {arg1}, {arg2}, {obj})')
        if what not in self.keys:
            return False

        if ServiceType.APP == arg1:
            return self.on_app(arg2, obj)
        elif ServiceType.SRS == arg1:
            return self.on_srs(arg2, obj)

        return False

In [34]:
main_loop = MainLooper()
main_loop.add_handler(ServiceStateMessageHandler())
main_loop.start()
main_loop.default_handler.send_message(MessageType.STATE, ServiceType.APP, StateType.RUNNING, 'test')

2022-02-21 11:05:37,597 - handle_message:37 - nbeasy - INFO - (2, 1, 1, test)


True

2022-02-21 11:05:37,599 - on_app:28 - nbeasy - INFO - app running


In [ ]:
pub.send_string('frepai.engine {"workdir":"/frepai/tmp/pipetest", "strides": [2,3], "batch_size": 10}')

In [35]:
a = {'pigeon': {'msgkey': 'factory:action.inference.queue', 'push_video_addr': 'https://frepai-1301930378.cos.ap-beijing.myqcloud.com/live/00232ee8876d/20220222/videos/20220222114544.mp4', 'created_at': '2022-02-2212:04:16', 'etag': 'aa68a9827f3db8f3f70851adc2f30ebd', 'major': 473160, 'mac_addr': '00232ee8876d', 'uuid': 'SSAC-292217-AAFFA', 'video_id': 0, 'actual_tally': 0, 'auto_tally': 0, 'file_name': '', 'data_set': 0, 'param_md5': '', 'embs_feat': '', 'embs_sims': '', 'user_id': 0}, 'video': 'https://frepai-1301930378.cos.ap-beijing.myqcloud.com/live/00232ee8876d/20220222/videos/20220222114544.mp4', 'constant_speed': False, 'median_filter': True, 'fully_periodic': False, 'batch_size': 20, 'temperature': 13.544, 'strides': [1], 'angle': 0, 'reg_factor': 1, 'detect_focus': False, 'focus_box': [0.754, 0.302, 0.93, 0.688], 'focus_box_repnum': 3, 'black_box': [0, 0, 0, 0], 'black_overlay': False, 'rm_still': True, 'area_rate_threshold': 0.001, 'best_stride_video': False, 'osd_sims': False, 'osd_feat': False, 'save_video': False, 'ef_is_send': False, 'ef_url': '', 'ef_alpha': 0.01, 'ef_beta': 0.7, 'ef_gamma': 0.8}
a

{'pigeon': {'msgkey': 'factory:action.inference.queue',
  'push_video_addr': 'https://frepai-1301930378.cos.ap-beijing.myqcloud.com/live/00232ee8876d/20220222/videos/20220222114544.mp4',
  'created_at': '2022-02-2212:04:16',
  'etag': 'aa68a9827f3db8f3f70851adc2f30ebd',
  'major': 473160,
  'mac_addr': '00232ee8876d',
  'uuid': 'SSAC-292217-AAFFA',
  'video_id': 0,
  'actual_tally': 0,
  'auto_tally': 0,
  'file_name': '',
  'data_set': 0,
  'param_md5': '',
  'embs_feat': '',
  'embs_sims': '',
  'user_id': 0},
 'video': 'https://frepai-1301930378.cos.ap-beijing.myqcloud.com/live/00232ee8876d/20220222/videos/20220222114544.mp4',
 'constant_speed': False,
 'median_filter': True,
 'fully_periodic': False,
 'batch_size': 20,
 'temperature': 13.544,
 'strides': [1],
 'angle': 0,
 'reg_factor': 1,
 'detect_focus': False,
 'focus_box': [0.754, 0.302, 0.93, 0.688],
 'focus_box_repnum': 3,
 'black_box': [0, 0, 0, 0],
 'black_overlay': False,
 'rm_still': True,
 'area_rate_threshold': 0.001,
 